# How to get started with Deep Learning in Jax (Part I)

If you have been developing your own deep learning models, you are probably familiar with the most popular deep learning frameworks out there: PyTorch and Tensorflow. Both frameworks are heavily used in organizations large and small and from research to production. Despite the popularity of PyTorch and Tensorflow, many research organizations are already moving on towards a new library, with the industry not lagging much behind. This new library is called [JAX](https://jax.readthedocs.io/en/latest/index.html) and its gaining a lot of momentum. If you are still deciding whether JAX is right for you, you might want to check out my other blogpost _[5 Reasons why you should choose JAX for your deep learning projects instead of PyTorch or Tensorflow]()_. If you have already decided that JAX is the right tool for you, but are looking for a way to get started, then I'm happy you are here. Because we'll be doing just that! So, grab a cup of coffee, sit up straight and let's get started.




## What is JAX and why should I use it?

So you have decided to explore JAX and see for yourself it is worth the effort to learn. Great! You open your browser, go to the official JAX documentation to read and get a better feeling of the benefits of JAX. In case you are familiar with either PyTorch or Tensorflow, you are probably expecting something in the lines of: _JAX: An open source machine learning framework that accelerates the path from research prototyping to production deployment._ But when you land on the homepage of JAX, you find something entirely different. It reads: _JAX is autograd and XLA brought together for high-performance numerical computing and machine learning research. It provides composable transformations of Python+NumPy programs: differentiate, vectorize, parallelize, Just-In-Time compile to GPU/TPU._ That sounds, well, complex... Let's try to make it a bit more human readable. What they are essentially saying is the following. JAX takes care of computating the gradients for you such that you can train your machine learning models using JAX. Ooh, and while we are at it, we enable you to run it on any compute type you like (think CPUs, GPUs and even TPUs). But wait! There is more. we even make it heighly efficient through vectorization, paralleization and Just-In-Time compilation. Is that it? Nope. What's even better is that you can leverage some of your existing knowledge as we provide you with a familiar, clean numpy-like interface! Wow. That sounds great! You probably understand now why this is a big deal. 

So now what? Now it's time to get you up and running with JAX as fast as we can! Ready, set, go!


## JAX: not your typical way to program in python


To understand how to work effectively with JAX, we must keep one attribute in mind, which is that JAX only works with pure functions. Pure functions are simply functions with no side effects. Basically, these functions are idompotent, meaning they always have the same effect under the same conditions. So lets see what this exactly means with some code and get a better grip on what works and what doesn't.

In [6]:
from jax import jit

n_pies = 0

@jit
def add_pies(pies_to_add: int):
    return n_pies + pies_to_add
print(f"Adding 10 pies to 0 pies = {add_pies(10)}")

Adding 10 pies to 0 pies = 10




Awesome 0 + 10 = 10, that's what we expected!
But now lets see what happens if we run the function again but change the global variable n_pies to 20. What you might expect is the following.
We currently have 20 pies, we add 10 pies, so we surely must end with 30 pies in the bank. Right?

In [9]:
n_pies = 20
print(f"Adding 10 pies to 20 pies = {add_pies(10)}")

Adding 10 pies to 20 pies = 10



Darn... That doesn't seem right. The last time I checked 10 + 20 should have been 30, but we are still getting our old result back?! What's going on?
What is happening is a combination of JIT-compiliation with functional programming :). On the first execution, JAX compiles your code and then runs it, making sure that every time your function is called after the first time, it will see some execution speedups. Basically, the second time it runs, it uses a cached compilation of your function. And this is exactly what causes your results to be off... The global variable n_pies has been cached and you introduced a non-pure function. So what can we do to turn our function into a pure function? We can add some more arguments :)

In [12]:

from jax import jit

@jit
def pure_add_pies(n_pies: int, pies_to_add: int):
    return n_pies + pies_to_add

print(f"Starting with 0 pies and adding 10 pies = {pure_add_pies(0, 10)}")
print(f"Starting with 20 pies and adding 10 pies = {pure_add_pies(20, 10)}")

Starting with 0 pies and adding 10 pies = 10
Starting with 20 pies and adding 10 pies = 30


That's more like it! Let's keep that in mind, JAX compiles your functions and uses the cached version in subsequent calls to speed up computations. As a result, we need to make sure our functions are pure functions to prevent any unexpected side-effects.


# Immutability
JAX arrays are immutable objects. If you are not familiar with the term immutable/immutability, it simply means that an object can't be changed once it has been created. The reason JAX objects are immutable has to with simplifying some of the functionality that JAX provides and numpy doesn't. To put it briefly, if you have immutable objects, it's easier to analyse programs and transformation. As a result, compilation is straingforward and we can more easily convert the user friendly numpy API interface (i.e. `import jax.numpy as jnp`) to the lower level `lax` API and compile to `XLA` (which stands for Accelerated Linear Algebra). As a result, JAX doesn't allow for object mutation. So how then do we change values of objects we have created? Lets first see what will not work:

In [3]:

import jax.numpy as jnp
import numpy as np

base = np.array([1, 2, 3, 4, 5])
base[0] = 100
print(base)
# >>> [100, 2, 3, 4, 5]

base_jax = jnp.array([1, 2, 3, 4, 5])
try:
    base_jax[0] = 100
except TypeError:
    print("object does not support item assignment. JAX arrays are immutable.")
print(base_jax)

[100   2   3   4   5]
object does not support item assignment. JAX arrays are immutable.
[1 2 3 4 5]


As expected. The object is immutable, so we can't assign a new value to that particular object. However, we can make it work. We just need to modify the code slightly. The interface that JAX uses to 'modify' an object at an index is as follows:

In [5]:

import jax.numpy as jnp
import numpy as np

base = np.array([1, 2, 3, 4, 5])
base[0] = 100
print(base)
# >>> [100, 2, 3, 4, 5]

base_jax = jnp.array([1, 2, 3, 4, 5])
# Notice the square brackets
# and the assignment to the variable updated_base (functional programming:))
updated_base = base_jax.at[0].set(100)
print(updated_base)
# >>> [100, 2, 3, 4, 5]

[100   2   3   4   5]
[100   2   3   4   5]


And that's it. We can slice it in the same way we were familiar doing with numpy, but just need to remember to use the `.at[slice]` interface combined with the `.set()` function. So far so good! We have been able to get a better grisp on pure functions and immutability. Now lets speed things up. and for that we'll need JIT. To the moon!!! (I'm so happy I found a way to include a crypto joke in this blogpost).


# Just-In-Time compilation: Speed it up!
Although how the name JAX came to be remains unclear, some argue the J in JAX stands for JIT, or Just-In-Time. What JIT does is it compiles your code to XLA (do you remember what it stands for? Accelerated Linear Algebra) so that it can be executed efficiently on the compute type that you use (e.g. GPUs, TPUs).

What JIT does, it compiles your function and merges operations together to run them more efficiently. There are some requirements that must be true though in order for this to work. For example, the shapes of your arrays must be static and known at compile time. But if you adhere to this, you can expect some improvements. For example, the following code, which has not been jitted, would run okay-ish.

In [12]:
import jax
import jax.numpy as jnp

def selu(x, alpha=1.67, lambda_=1.05):
  return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()

2.15 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


You should expect to see an average execution time of around 3 milliseconds and has a standard devation of 300 microseconds. Note the mean is in milliseconds, while the standard devation is in microseconds. If you don't know what a standard deviation is, no worries. Just have a look at the average for now. Now lets JIT the exact same function and see if we get any improvements.

In [13]:
selu_jit = jax.jit(selu)

# Warm up
selu_jit(x).block_until_ready()

%timeit selu_jit(x).block_until_ready()


583 µs ± 38.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Do you see the speedup? The average processing time is not in milliseconds anymore, it's in microseconds! Not bad to achieve such a speed up so easily! Remember what is happening. We first need to call the function once, this makes sure it is compiled and operations are fused. After that, for each execution we get an performance boost.


# Vectorization map also known as vmap

You might be familiar with map and apply in python. Well, that's basically Vmap or Vectorizedmap in JAX. Ofcourse, in reality there is plenty of stuff abstracted away from you to execute `vmap` efficiently, but thinking in terms of mapping a function and applying it simultaneously to each element in an array might do it for now. As we are just getting started with JAX, don't worry too much about the details for now. You can worry about that once you fully understand the fundamentals.

So what `vmap` does, it takes your code, or your function, and maps it over the input parameters. Now why do we need this in the first place? The main reason is that we don't need to rewrite our code to work with difference input shapes which you will often encounter in deep learning. For example, how do you handle a single example vs a batch of examples. With JAX, we can easily achieve that with the `vmap` function. But first, as always, lets see what doesn't work and how `vmap` fixes it.


In [18]:
from jax import jit, vmap
from jax import random


def dotproduct(w, x):
    return jnp.dot(w, x)

# dotproduct works on a single sample, but we want to apply it to all samples in the batch.

keygen = random.PRNGKey(0)
feature = jnp.array([1.5, 2.5])
weights = random.normal(keygen, shape=(20, 2)) # hidden layer with 20 neurons and 2 inputs
pred = dotproduct(weights, feature) #works :)


(20,)

In [21]:

features = random.normal(keygen, shape=(5, 2)) # batch of 5 samples with two features
# now we want to apply it to all samples in the batch
dotproduct = jit(dotproduct)
try:
    pred = dotproduct(weights, features)
except TypeError:
    print("TypeError: Incompatible shapes for dot: got (20, 2) and (5, 2).")


TypeError: Incompatible shapes for dot: got (20, 2) and (5, 2).


With vmap, we can easily write our code for a single sample and than map every sample to that single function, like so:

In [26]:
batch_dim_weights = None
batch_dim_features = 0
vdot_product = vmap(dotproduct, in_axes=(batch_dim_weights, batch_dim_features))

preds = vdot_product(weights, features)
preds.shape

(5, 20)

One thing to be aware of is the `in_axes` argument. This tells JAX along which parameters it should paralellize. `in_axes` is a tuple which is always the size equal to number of input parameters of the function. In our case this means that we have two arguments in `dotproduct`, so our in_axis should be a tuple of length two. Secondly, we need to specify to which axis to apply the vmap to. As our weights don't have a batch dimension, we set this to None (i.e. `batch_dim_weights = None`). Our features do have a batch dimension in this case, which is the the first axis, so `batch_dim_features = 0`. And there you have it. We just vectorized our dotproduct implementation. Great!

## The missing ML piece: Gradients

We got most of basics covered by now, but there is still a single piece missing, which is computing gradients. Even this is handled with easy by JAX. It is similar to [Autograd](https://github.com/HIPS/autograd), whose main developers have stopped working on Autograd and are now working on JAX. With `jax.grad` we can compute the gradient of a function and even take the gradient of gradients. Lets see what it looks like for an easy function like $f(x) = x^2$. We would expect $f'(x) = \frac{f(x)}{dx} = 2x$ and $f''(x) = \frac{f'(x)}{dx} = 2$ as our results.

In [34]:
from jax import grad

def forward(x: int):
    return x ** 2

derivative_forward = grad(forward)
derivate_derivative_forward = grad(grad(forward))

x_value = 10.0
result_function_forward = forward(x_value)
result_derivative_forward = derivative_forward(x_value)
result_derivative_derivative_forward = derivate_derivative_forward(x_value)


print(f"f(10) = {result_function_forward} and f'(10) = {result_derivative_forward} and f''(10) = {result_derivative_derivative_forward}")

f(10) = 100.0 and f'(10) = 20.0 and f''(10) = 2.0


And there you go. Precisely in line with our expectations.


## Conclusion

Although established frameworks like PyTorch and Tensorflow have dominated the machine learning landscape over the last few years, with JAX the status quo is challenged as it provides some powerful capabilities. When you get started with JAX, you might need to wrap your head around some of the basics (like immutability and pure functions), but once you've got that covered, its a joy to work with. Want to run the examples yourself, check out this notebook: [ml_with_jax_part_I.ipynb](https://github.com/Sruinard/machine_learning_novice/blob/main/introduction_to_deep_learning_with_jax/ml_with_jax_part_1.ipynb). Still questioning whether JAX is something for you, checkout my other blogpost: [5 Reasons why you should choose JAX for your deep learning projects instead of PyTorch or Tensorflow](). Ready to train your first machine learning model with jax.numpy, checkout my other notebook: [ml_with_jax_part_II.ipynb](https://github.com/Sruinard/machine_learning_novice/blob/main/introduction_to_deep_learning_with_jax/ml_with_jax_part_2.ipynb)

## Connect, learn and contribute to help yourself and others land a job in the AI space

Looking for a way to contribute or learn more about AI/ML, connect with me on medium:
- LinkedIn: [https://www.linkedin.com/in/stefruinard/]()
- Medium: [https://medium.com/@stefruinard]()
- GitHub: [https://github.com/Sruinard]()

## Contributors:
###### Submit a Pull Request or reach out on LinkedIn and become a recognized contributor :)